In [1]:
import numpy as np
from scipy.sparse import coo_matrix
import os

In [2]:
def write_alist_file(H: coo_matrix, filename: str):

    if os.path.isfile(filename):
        choice = input(f"Warning file: {filename} exists!!! Overwrite y/n [n]?'")
        if (choice != "y") or (choice != "Y"):
            return
    
    try:
        M, N = H.shape

        sum_dv = np.array(H.sum(axis=0)).flatten().astype(int)
        sum_dc = np.array(H.sum(axis=1)).flatten().astype(int)

        max_dv = sum_dv.max()
        max_dc = sum_dc.max()

        with open(filename, "w") as f:
            # write dimensions of H
            f.write(f"{N} {M}\n")

            # write max weights
            f.write(f"{max_dv} {max_dc}\n")

            f.write(" ".join(map(str, sum_dv)) + "\n")
            f.write(" ".join(map(str, sum_dc)) + "\n")
            f.close()

        ind = H.row  
        col = H.col

        col = col * max_dv

        for _ in range(max_dv):
            diff = np.concatenate(([1], col[1:] - col[:-1]))
            col[diff == 0] += 1

        dvidx = np.zeros((max_dv, N), dtype=int)
        dvidx[col, ind] = ind

        with open(filename, "a") as f:
            for row in dvidx.T:
                f.write(" ".join(map(str, row)) + "\n")
                f.close()


        ind = H.col
        row = H.row

        row = row * max_dc

        for _ in range(max_dc):
            diff = np.concatenate(([1], row[1:] - row[:-1]))
            row[diff == 0] += 1

        dcidx = np.zeros((max_dc, M), dtype=int)
        dcidx[row, ind] = ind

        with open(filename, "a") as f:
            for row in dcidx.T:
                f.write(" ".join(map(str, row)) + "\n")
                f.close()
   
    except Exception as e:
        print(e)